In [ ]:
import pandas as pd
from pulp import LpVariable, LpProblem, LpMaximize, LpStatus, value, lpSum


data_all = pd.read_csv('')
data = data_all[data_all['GW'] == 1]


positions = pd.get_dummies(data.position)
teams =     pd.get_dummies(data.team)

names =   [data.name[i] for i in range(len(data))]
points =  [data.total_points[i] for i in range(len(data))]
pos_def = [positions.DEF[i] for i in range(len(positions))]
pos_fwd = [positions.FWD[i] for i in range(len(positions))]
pos_gk =  [positions.GK[i] for i in range(len(positions))]
pos_mid = [positions.MID[i] for i in range(len(positions))]
value =   [data.value[i] for i in range(len(data))]

# Problem
prob = LpProblem('Problem', LpMaximize)

# Variables
x1 = [LpVariable('x' + str(i), cat='Binary') for i in range(len(data))]



#  Objective
prob += (lpSum(x1[i] * points[i] for i in range(len(data))))



# Constraints
prob += lpSum(x1[i] * value[i] for i in range(len(data))) <= 1000  # Budget
prob += lpSum(x1[i] * pos_gk[i] for i in range(len(data))) == 2  # Goalkeepers
prob += lpSum(x1[i] * pos_fwd[i] for i in range(len(data))) == 3  # Forwards
prob += lpSum(x1[i] * pos_mid[i] for i in range(len(data))) == 5  # Midfielders
prob += lpSum(x1[i] * pos_def[i] for i in range(len(data))) == 5  # Defenders


for team in teams.columns:
    prob += lpSum(x1[i] * teams[team].values[i] for i in range(len(data))) <= 3 # Teammate restriction 
    

status = prob.solve()

print(f"status={LpStatus[status]}")


In [ ]:
team_pts = []
team_cost = []
team_roster = []
team_pos = []
team_team = []
count = 0

for var in sorted(prob.variables(), key=lambda v: int(v.name[1:])):
    if var.varValue == 1:
        team_roster.append(names[count])
        team_pos.append(data.position[count])
        team_team.append(data.team[count])
        team_pts.append(points[count])
        team_cost.append(value[count])
    count += 1
    
print('points:',sum(team_pts))
print('cost:', sum(team_cost))

In [ ]:
roster_dict = {'player' : team_roster, 'position' : team_pos, 'team' : team_team, 'points' : team_pts, 'cost' : team_cost}


opt_roster = pd.DataFrame(roster_dict)
opt_roster